## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import os
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np

# Import API key
from config import g_c_api_key

# Configure gmaps API key
gmaps.configure(api_key=g_c_api_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df=pd.read_csv("..\\Weather_Database\\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,0,Ushuaia,-54.8000,-68.3000,56.93,41,82,9.42,broken clouds,AR,2021-12-04 20:11:58
1,1,Povenets,62.8487,34.8262,6.01,82,12,12.26,few clouds,RU,2021-12-04 20:12:32
2,2,Atuona,-9.8000,-139.0333,78.35,74,4,17.49,clear sky,PF,2021-12-04 20:02:42
3,3,Rikitea,-23.1203,-134.9692,75.81,73,18,12.08,few clouds,PF,2021-12-04 20:08:48
4,4,Severo-Kurilsk,50.6789,156.1250,31.37,75,76,27.60,broken clouds,RU,2021-12-04 20:08:58


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
2,2,Atuona,-9.8000,-139.0333,78.35,74,4,17.49,clear sky,PF,2021-12-04 20:02:42
3,3,Rikitea,-23.1203,-134.9692,75.81,73,18,12.08,few clouds,PF,2021-12-04 20:08:48
8,8,Hobyo,5.3505,48.5268,79.16,73,50,16.33,scattered clouds,SO,2021-12-04 20:09:55
10,10,Castro,-24.7911,-50.0119,79.61,43,12,5.10,few clouds,BR,2021-12-04 20:12:33
11,11,Mopti,14.6667,-4.0000,83.82,10,100,10.11,overcast clouds,ML,2021-12-04 20:12:33
14,14,Sinnamary,5.3833,-52.9500,78.31,89,84,7.74,broken clouds,GF,2021-12-04 20:09:50
16,16,Raga,8.4596,25.6780,81.12,27,35,4.29,scattered clouds,SS,2021-12-04 20:12:34
30,30,Bengkulu,-3.8004,102.2655,75.72,90,100,9.19,light rain,ID,2021-12-04 20:02:45
40,40,Hithadhoo,-0.6000,73.0833,81.19,79,69,11.83,broken clouds,MV,2021-12-04 20:08:28
43,43,Hervey Bay,-25.2986,152.8535,75.04,77,59,8.86,broken clouds,AU,2021-12-04 20:11:49


In [13]:
# 4a. Determine if there are any empty rows.
#preferred_cities_df.count()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Country                1
Date                   0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
#clean_df.count()

In [15]:
clean_df.count()

City_ID                212
City                   212
Lat                    212
Lng                    212
Max Temp               212
Humidity               212
Cloudiness             212
Wind Speed             212
Current Description    212
Country                212
Date                   212
dtype: int64

In [16]:
clean_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Country                0
Date                   0
dtype: int64

In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.35,clear sky,-9.8000,-139.0333,
3,Rikitea,PF,75.81,few clouds,-23.1203,-134.9692,
8,Hobyo,SO,79.16,scattered clouds,5.3505,48.5268,
10,Castro,BR,79.61,few clouds,-24.7911,-50.0119,
11,Mopti,ML,83.82,overcast clouds,14.6667,-4.0000,
14,Sinnamary,GF,78.31,broken clouds,5.3833,-52.9500,
16,Raga,SS,81.12,scattered clouds,8.4596,25.6780,
30,Bengkulu,ID,75.72,light rain,-3.8004,102.2655,
40,Hithadhoo,MV,81.19,broken clouds,-0.6000,73.0833,
43,Hervey Bay,AU,75.04,broken clouds,-25.2986,152.8535,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_c_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping this one")

Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one
Hotel not found...skipping this one


In [24]:
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.35,clear sky,-9.8000,-139.0333,Villa Enata
3,Rikitea,PF,75.81,few clouds,-23.1203,-134.9692,People ThankYou
8,Hobyo,SO,79.16,scattered clouds,5.3505,48.5268,Halane Hotel and Restaurent
10,Castro,BR,79.61,few clouds,-24.7911,-50.0119,CHACARA BAILLY
11,Mopti,ML,83.82,overcast clouds,14.6667,-4.0000,
14,Sinnamary,GF,78.31,broken clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
16,Raga,SS,81.12,scattered clouds,8.4596,25.6780,
30,Bengkulu,ID,75.72,light rain,-3.8004,102.2655,Grage Hotel Bengkulu
40,Hithadhoo,MV,81.19,broken clouds,-0.6000,73.0833,Scoop Guest House
43,Hervey Bay,AU,75.04,broken clouds,-25.2986,152.8535,Shelly Bay Resort


In [28]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('',np.nan,inplace=True)
clean_hotel_df=hotel_df.dropna()


In [30]:
clean_hotel_df.tail(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
624,Mount Pleasant,US,75.18,clear sky,32.7941,-79.8626,Best Western Patriots Point
627,Vanimo,PG,77.54,light rain,-2.6741,141.3028,Vahmoneh Lodge Vanimo
628,Balikpapan,ID,75.90,light rain,-1.2675,116.8289,ibis Balikpapan
629,Meulaboh,ID,76.12,overcast clouds,4.1363,96.1285,Kost Pria
633,Axim,GH,80.26,few clouds,4.8699,-2.2405,Axim Beach Resort And Spa
639,Conceicao Das Alagoas,BR,85.78,broken clouds,-19.9147,-48.3883,Bouganville Hotel
641,Belawan,ID,75.49,broken clouds,3.7755,98.6832,Rumah Mamak Bapak M.Radial
646,Eyl,SO,77.23,overcast clouds,7.9803,49.8164,Amiin Hotel
649,Mercedes,AR,80.04,overcast clouds,-34.6515,-59.4307,Posada del Inti Centro
655,Quatre Cocos,MU,78.10,light rain,-20.2078,57.7625,Sunrise Attitude Hotel


In [31]:
clean_hotel_df.count()

City                   189
Country                189
Max Temp               189
Current Description    189
Lat                    189
Lng                    189
Hotel Name             189
dtype: int64

In [32]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))